In order to use Graph Grammar user have to build node and rule vocabularies the instances of the NodeVocabulary and RuleVocabulary classes and fill them with nodes and rules for the graph. This tutorial provides the description of the creating nodes and rules process.  

Before starting to build your vocabularies import all necessary libs standard and from the rostok package 

In [1]:
from cmath import sqrt

import numpy as np

from rostok.block_builder.node_render import (ChronoRevolveJoint,
                                              ChronoTransform, FlatChronoBody,
                                              LinkChronoBody, MountChronoBody)
from rostok.block_builder.transform_srtucture import FrameTransform
from rostok.graph_grammar.node import ROOT, BlockWrapper, GraphGrammar
from rostok.graph_grammar.node_vocabulary import NodeVocabulary
from rostok.graph_grammar.rule_vocabulary import RuleVocabulary

Create and instance of the [class](https://licaibeerlab.github.io/rostok/modules/graph_grammar/node_rule_vocabulary.html#rostok.graph_grammar.node_vocabulary.NodeVocabulary) `NodeVocabulary` and add the imported node ROOT to your vocabulary, the ROOT node is special starting point that have to be in any node vocabulary. 

There are two types of nodes terminal and non-terminal. The non-terminal nodes describe a part of the robot in general without physical properties, for example node "J" represents joints. The nodes are the objects of the class `Node` and terminal nodes actually have only one attribute - label. So one should plan how many structural nodes is needed in the rules. Here we have 14 nodes.

In [2]:
node_vocab = NodeVocabulary()
node_vocab.add_node(ROOT)
node_vocab.create_node("J")
node_vocab.create_node("L")
node_vocab.create_node("F")
node_vocab.create_node("M")
node_vocab.create_node("EF")
node_vocab.create_node("EM")

The second type of nodes are terminal nodes - they represent the physical properties of the nodes in the robot. The physical properties of a node are defined through an instance of the `BlockWrapper` class. To construct the `BlockWrapper` object one have to chose the type of the node by choosing one of the classes from node_render module and set the required for that class parameters.

Here for a joint node we choose `ChronoRevolveJoint` set the rotation around Z axis and set the type of the control - `TORQUE`.


In [3]:
type_of_input = ChronoRevolveJoint.InputType.TORQUE
# Joints
revolve1 = BlockWrapper(ChronoRevolveJoint, ChronoRevolveJoint.Axis.Z, type_of_input)
node_vocab.create_node(label="J1", is_terminal=True, block_wrapper=revolve1)

For each non-terminal nodes one can have several terminal nodes - several details with specific physical properties that have the same function and hence the same non-terminal node. Here we have two palms with different sizes specified with width. The `FlatChronoBody` represents the fixed part of the mechanism and has the form of the box. Here we only set the size of the box and other parameters remain default. The total list of parameters can be seen in the class [documentation](https://licaibeerlab.github.io/rostok/modules/graph_grammar/graph_builder.html#rostok.block_builder.node_render.FlatChronoBody).

In [4]:
flat1 = BlockWrapper(FlatChronoBody, width=0.4, length=0.2)
flat2 = BlockWrapper(FlatChronoBody, width=0.7, length=0.2)
node_vocab.create_node(label="F1", is_terminal=True, block_wrapper=flat1)
node_vocab.create_node(label="F2", is_terminal=True, block_wrapper=flat2)

Next are the links that represents the parts of the fingers. Here we create two different links specified by lengths using class `LinkChronoBody`. It also has more parameters that can be seen in class [documentation](https://licaibeerlab.github.io/rostok/modules/graph_grammar/graph_builder.html#rostok.block_builder.node_render.LinkChronoBody).

In [5]:
link1 = BlockWrapper(LinkChronoBody, length=0.6)
link2 = BlockWrapper(LinkChronoBody, length=0.4)
node_vocab.create_node(label="L1", is_terminal=True, block_wrapper=link1)
node_vocab.create_node(label="L2", is_terminal=True, block_wrapper=link2)

We also add a fingertip that would end the fingers of the grasping mechanism, setting them with two sizes. Other parameters see in documentation.

In [6]:
u1 = BlockWrapper(MountChronoBody, width=0.1, length=0.1)
node_vocab.create_node(label="U1", is_terminal=True, block_wrapper=u1)

Before we had nodes that represented the physical bodies. Here we consider the special case - nodes that represent the coordinates of the base of a finger on the palm. In that set of nodes it is the most difficult part. First we create function that transforms angle in degrees into the rotation that can be set for `Pychrono` object.  


For each palm we create several movements using `FrameTransform` object, that consists of the translation 



In [7]:
# Transforms
MOVE_ZX_PLUS = FrameTransform([0.3, 0, 0.3], [1, 0, 0, 0])
MOVE_ZX_MINUS = FrameTransform([-0.3, 0, -0.3], [1, 0, 0, 0])

MOVE_X_PLUS = FrameTransform([0.3, 0, 0.], [1, 0, 0, 0])
MOVE_Z_PLUS_X_MINUS = FrameTransform([-0.3, 0, 0.3], [1, 0, 0, 0])

For each movement we also create a `BlockWrapper` object 

transform_mzx_plus = BlockWrapper(ChronoTransform, MOVE_ZX_PLUS)
transform_mzx_minus = BlockWrapper(ChronoTransform, MOVE_ZX_MINUS)
transform_mx_plus = BlockWrapper(ChronoTransform, MOVE_X_PLUS)
transform_mz_plus_x_minus = BlockWrapper(ChronoTransform, MOVE_Z_PLUS_X_MINUS)

In [8]:
node_vocab.create_node(label="T1", is_terminal=True, block_wrapper=transform_mx_plus)
node_vocab.create_node(label="T2", is_terminal=True, block_wrapper=transform_mz_plus_x_minus)
node_vocab.create_node(label="T3", is_terminal=True, block_wrapper=transform_mzx_plus)
node_vocab.create_node(label="T4", is_terminal=True, block_wrapper=transform_mzx_minus)

NameError: name 'transform_mx_plus' is not defined